In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-19 01:52:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.68MB/s    in 0.2s    

2022-09-19 01:52:59 (5.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [7]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
#create df with total votes greater than or equal to 20
filtered_vine_df=vine_df.filter("total_votes>=20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [12]:
helpful_review_df=filtered_vine_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [16]:
paid_review_df=helpful_review_df.filter("vine=='Y'")
paid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|
| R9PYAUDIBJVEC|          4|           12|         22|   Y|                N|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|
| R86Z11D4CWOFM|          4|           32|         36|   Y|                N|
| RNP01HW9YISJO|          4|           20|         23|   Y|                N|
|R3KLACA6LCDZ0S|          3|           21|         22|   Y|                N|
| RZEQYOT2RE0N7|          4|           75|         80|   Y|     

In [17]:
unpaid_review_df=helpful_review_df.filter("vine=='N'")
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [19]:
total_num_helpful_reviews = helpful_review_df.count()
total_num_helpful_reviews

65581

In [20]:
# paid total number of reviews
total_paid_reviews = paid_review_df.count()
total_paid_reviews

613

In [21]:
# unpaid total number of reviews
total_unpaid_reviews = unpaid_review_df.count()
total_unpaid_reviews

64968

In [22]:
# paid 5-star reviews
paid_five_star_reviews = paid_review_df.filter(paid_review_df.star_rating == 5).count()
paid_five_star_reviews

222

In [23]:
# unpaid 5-star reviews
unpaid_five_star_reviews = unpaid_review_df.filter(unpaid_review_df.star_rating == 5).count()
unpaid_five_star_reviews

30543

In [24]:
# paid 5-star reviews percentage
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

36.215334420880914

In [25]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

47.01237532323606